### Patch sklearn (has to be done before importing anything from sklearn)

In [ ]:
from sklearnex import patch_sklearn

patch_sklearn()

### Load data

In [ ]:
from data_manager import DataManager

X_train, y_train = DataManager.get_train()
X_val, y_val = DataManager.get_validation()
X_test, y_test = DataManager.get_test()

### Helper functions

In [ ]:
from formatter import export
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error


def get_error(model, X, y):
    y_pred = model.predict(X)

    return mean_absolute_error(y, y_pred)

def get_pva(y_true, y_pred, model_name):
    plt.xlabel('Actual value')
    plt.ylabel('Predicted value')
    plt.scatter(y_true, y_pred)
    export(f'{model_name}_pva')
    plt.close()
    plt.cla()
    plt.clf()


## Benchmark

In [ ]:
from models import models_by_name
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from timeit import default_timer as timer


for model_name, get_model in models_by_name.items():
    if model_name != 'custom_mlp': continue
    print(f'\n[*] Going with {model_name}...')

    print('\tFitting training...')
    model = get_model()
    model.fit(X_train, y_train)
    print(f'\t\tTrain error: {get_error(model, X_train, y_train):.5f}')
    print(f'\t\tEvaluation error: {get_error(model, X_val, y_val):.5f}')

    print('\tFitting training + validation')
    model = get_model()

    start = timer()
    model.fit(pd.concat((X_train, X_val)), np.concatenate((y_train, y_val)))
    end = timer()
    training_time = end - start

    start = timer()
    y_pred = model.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_pred)
    end = timer()
    prediction_time = end - start
    get_pva(y_test, y_pred, model_name)

    print(f'\t\tTest error: {test_mae:.5f}')
    print(f'\t\tTraining time: {training_time:.2f} s')
    print(f'\t\tPrediction time: {prediction_time:.2f} s')